In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.models import ModelCatalog


import ray
from ray import tune
from ray.tune import track

import math
import gym

from gym_compete_to_rllib import created_envs, env_name, create_env, env_name_rllib

import os
os.environ['DISPLAY'] = ':0'

In [2]:
import os
dir(os)

['CLD_CONTINUED',
 'CLD_DUMPED',
 'CLD_EXITED',
 'CLD_TRAPPED',
 'DirEntry',
 'EX_CANTCREAT',
 'EX_CONFIG',
 'EX_DATAERR',
 'EX_IOERR',
 'EX_NOHOST',
 'EX_NOINPUT',
 'EX_NOPERM',
 'EX_NOUSER',
 'EX_OK',
 'EX_OSERR',
 'EX_OSFILE',
 'EX_PROTOCOL',
 'EX_SOFTWARE',
 'EX_TEMPFAIL',
 'EX_UNAVAILABLE',
 'EX_USAGE',
 'F_LOCK',
 'F_OK',
 'F_TEST',
 'F_TLOCK',
 'F_ULOCK',
 'MutableMapping',
 'NGROUPS_MAX',
 'O_ACCMODE',
 'O_APPEND',
 'O_ASYNC',
 'O_CLOEXEC',
 'O_CREAT',
 'O_DIRECT',
 'O_DIRECTORY',
 'O_DSYNC',
 'O_EXCL',
 'O_LARGEFILE',
 'O_NDELAY',
 'O_NOATIME',
 'O_NOCTTY',
 'O_NOFOLLOW',
 'O_NONBLOCK',
 'O_RDONLY',
 'O_RDWR',
 'O_RSYNC',
 'O_SYNC',
 'O_TRUNC',
 'O_WRONLY',
 'POSIX_FADV_DONTNEED',
 'POSIX_FADV_NOREUSE',
 'POSIX_FADV_NORMAL',
 'POSIX_FADV_RANDOM',
 'POSIX_FADV_SEQUENTIAL',
 'POSIX_FADV_WILLNEED',
 'PRIO_PGRP',
 'PRIO_PROCESS',
 'PRIO_USER',
 'P_ALL',
 'P_NOWAIT',
 'P_NOWAITO',
 'P_PGID',
 'P_PID',
 'P_WAIT',
 'PathLike',
 'RTLD_DEEPBIND',
 'RTLD_GLOBAL',
 'RTLD_LAZY',
 'RTLD_LO

In [3]:
env_cls = create_env
env_config = {'with_video': True}

def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "player_%d"

    def get_agent_config(agent_id):
        agent_config = (PPOTFPolicy, obs_space, act_space, {
            'model': {
                        "custom_model": "GymCompetePretrainedModel",
                        "custom_model_config": {
                            "agent_id": agent_id - 1,
                            "env_name": env_name,
                            "model_config": {},
                            "name": "model_%s" % (agent_id - 1)
                        },           
                        
                    },
            
            "framework": "tfe",
        })
        
        return agent_config
        
#         agent_config = (PPOTFPolicy, obs_space, act_space, {
#                     "model": {
#                         "use_lstm": False,
#                         "fcnet_hiddens": [64, 64],
#                         #"custom_action_dist": "DiagGaussian",
#                     },
#                     "framework": "tfe",
#                 })
        
#         return agent_config

    N_POLICIES = 2

    policies = {policy_template % i: get_agent_config(i) for i in range(1, 1  + N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player_1", "player_2"]
        agent_ids = ["player_1", "player_2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_name_rllib,
        "env_config": env_config,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
       "train_batch_size": config['train_batch_size'],
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    print("Using config")
    print(config)
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

def train(trainer, stop_iters, do_track=True):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        if do_track:
            track.log(**results)
    o = trainer.save_to_object()
    return o

trainer = None

def train_one(config, restore_state=None, do_track=True):
    print(config)
    rl_config = build_trainer_config(restore_state=restore_state,
                              train_policies=config['train_policies'],
                              config=config)
    global trainer
    trainer = build_trainer(restore_state=None, config=rl_config)
    train(trainer, config['train_steps'], do_track=do_track)


# try changing learning rate
config = {'train_batch_size': 128}

config['train_steps'] = 10
config['train_policies'] = [] #['player_1', 'player_2']
config['num_workers'] = 3

In [ ]:
train_one(config, do_track=False)

{'train_batch_size': 128, 'train_steps': 10, 'train_policies': [], 'num_workers': 3}


GLFW error: 65544, desc: b'X11: RandR gamma ramp support seems broken'


Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
{'env_name': 'multicomp/SumoAnts-v0', 'agent_a_type': 'zoo', 'agent_a_path': '1', 'agent_b_type': 'zoo', 'agent_b_path': '2', 'record_traj': False, 'record_traj_params': {'save_dir': 'data/trajectories', 'agent_indices': None}, 'transparent_params': None, 'num_env': 1, 'episodes': 20, 'timesteps': None, 'render': True, 'videos': False, 'video_params': {'save_dir': None, 'single_file': True, 'annotated': True, 'annotation_params': {'camera_config': 'default', 'short_labels': False, 'res

Install gputil for GPU system monitoring.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for

In [ ]:
from load_gym_compete_policy import get_policy_value_nets

In [ ]:
obs = env_cls(env_config).reset()

In [ ]:
acts = [trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)]

In [ ]:
np.mean(acts, axis=0)

In [ ]:
nets = get_policy_value_nets(env_name, 0)
policy_net_orig = nets['policy_mean_logstd_flat']

In [ ]:
m = trainer.get_policy('player_1').model
m.policy_net(obs['player_1'].reshape(1, -1))

In [ ]:
nets['policy_mean_logstd'](obs['player_1'].reshape(1, -1))

In [ ]:
mean, logstd = tf.split(policy_net_orig(obs['player_1'].reshape(1, -1)), 2, axis=1)

In [ ]:
np.mean([np.clip(np.random.normal(loc=mean[0], scale=np.exp(logstd[0]), size=(17,)), -0.4, 0.4) for _ in range(1000)], axis=0)

In [ ]:
np.mean([trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)], axis=0)

In [ ]:
trainer.compute_action(obs['player_1'], policy_id='player_1')

In [ ]:
env_cls(env_config).action_space.high